In [2]:
import pandas as pd

In [3]:
import numpy as np

In [201]:
import math

In [93]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

In [4]:
train = pd.read_csv("train_2v.csv")

In [5]:
train.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,30669,Male,3.0,0,0,No,children,Rural,95.12,18.0,NaN,0
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
2,16523,Female,8.0,0,0,No,Private,Urban,110.89,17.6,NaN,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
4,46136,Male,14.0,0,0,No,Never_worked,Rural,161.28,19.1,NaN,0


In [6]:
test = pd.read_csv("test_2v.csv")

In [7]:
test.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
2,14152,Female,14.0,0,0,No,children,Rural,95.16,21.2,NaN
3,12997,Male,28.0,0,0,No,Private,Urban,94.76,23.4,NaN
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked


In [8]:
train.isna().apply(pd.Series.value_counts)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
False,43400.0,43400.0,43400.0,43400.0,43400.0,43400.0,43400.0,43400.0,43400.0,41938,30108,43400.0
True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1462,13292,NaN


In [56]:
train = train.dropna()
train = train.drop(columns="id")

In [57]:
test.isna().apply(pd.Series.value_counts)

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
False,12423,12423,12423,12423,12423,12423,12423,12423,12423,12423,12423


In [58]:
test = test.dropna()
test = test.drop(columns="id")

In [59]:
def split_dataset(data, feature, value):
    return data[data[feature] < value], data[data[feature] >= value]

In [60]:
less_than, greater_than = split_dataset(train, "stroke", 1)

In [61]:
less_than.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
3,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
6,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0


In [62]:
greater_than.head()

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
63,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
141,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
257,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
264,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1
288,Male,81.0,0,0,Yes,Private,Urban,186.21,29.0,formerly smoked,1


In [202]:
def calculate_gini(data_a, data_b, binary_class):
    total_a = data_a[binary_class].count()
    class_counts_a = data_a[binary_class].value_counts().tolist()
    total_b = data_a[binary_class].count()
    class_counts_b = data_a[binary_class].value_counts().tolist()
    total_samples = total_a + total_b
    gini_index_a = math.inf
    gini_index_b = math.inf
    if total_a != 0:
        gini_index_a = (1.0 - sum([(count / total_a)**2 for count in class_counts_a])) * (total_a/total_samples)
    if total_b != 0:
        gini_index_b = (1.0 - sum([(count / total_b)**2 for count in class_counts_b])) * (total_b/total_samples)
    # print("a:", gini_index_a)
    # print("b:", gini_index_b)
    return gini_index_a + gini_index_b

In [203]:
test_gini_df_a = pd.DataFrame([[1, 1], [1, 0]])
test_gini_df_b = pd.DataFrame([[1, 1], [1, 1]])
calculate_gini(test_gini_df_a, test_gini_df_b, 1)

test_gini_df_a = pd.DataFrame([[1, 0], [1, 0]])
test_gini_df_b = pd.DataFrame([[1, 1], [1, 1]])
calculate_gini(test_gini_df_a, test_gini_df_b, 1)

0.0

In [209]:
def calculate_optimal_split(data, binary_class):
    lowest_gini = 999
    optimal = None
    for feature, col in data.iteritems():
        if is_string_dtype(data[feature]):
            continue
        if feature == binary_class:
            continue
        print(feature)
        for i, row in data.iterrows():
            low_data,high_data = split_dataset(data, feature, row[feature])
            new_gini = calculate_gini(low_data, high_data, binary_class)
            #print(new_gini)
            if new_gini < lowest_gini:
                optimal = (feature, row[feature], low_data, high_data)
                lowest_gini = new_gini
    return optimal

In [193]:
result = calculate_optimal_split(train.head(1000), "stroke")

age
hypertension
heart_disease
avg_glucose_level
bmi


In [194]:
feature, val, lo, hi = result

In [195]:
print(feature, val)

bmi 19.7


In [221]:
class DecisionNode:
    def __init__(self, feature, val, leaf=False):
        self.feature = feature
        self.val = val
        self.leaf = leaf
        self.left = None
        self.right = None
    
    def predict(self,data_row):
        if self.leaf:
            return self.val
        if data_row[self.feature] < self.val:
            return self.left.predict(data_row)
        else:
            return self.right.predict(data_row)

In [170]:
train["stroke"].value_counts().idxmin()

1

In [ ]:
DEPTH_LIMIT = 12
def build_tree(data, binary_class, depth=0):
    print(data[binary_class].value_counts())
    decision = data[binary_class].value_counts().idxmax()

    if depth >= DEPTH_LIMIT or (data.shape[0] < 4):
        root = DecisionNode(None, decision, leaf=True)
        return root
    else:
        feature, val, lo, hi = calculate_optimal_split(data, binary_class)
        root = DecisionNode(feature, val)
        if lo.shape[0] == 0:
            root.left = DecisionNode(None, decision, leaf=True)
        else:
            root.left = build_tree(lo, binary_class, depth + 1)

        if hi.shape[0] == 0:
            root.right = DecisionNode(None, decision, leaf=True)
        else:
            root.right = build_tree(hi, binary_class, depth + 1)
        return root

In [223]:
root = build_tree(train.head(500), "stroke")

0    490
1     10
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    127
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    28
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    18
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    11
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    2
Name: stroke, dtype: int64
0    9
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    4
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    2
Name: stroke, dtype: int64
0    2
Name: stroke, dtype: int64
0    5
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    3
Name: stroke, dtype: int64
0    2
Name: stroke, dtype: int64
0    7
Name: stroke, dtype: int64
age
hypertension
heart_disease
avg_glucose_level
bmi
0    4
Name: stroke, dtype: int

In [169]:
train.head(300)["stroke"].value_counts()

0    293
1      7
Name: stroke, dtype: int64

In [149]:
def print_tree(node, depth=0):
    if not node.leaf:
        print('%s[X%s < %.3f]' % ((depth*' ', node.feature, node.val)))
        print_tree(node.left, depth+1)
        print_tree(node.right, depth+1)
    else:
        print('%s[%s]' % ((depth*' ', node.val)))

In [213]:
print_tree(root)

[Xage < 32.000]
 [Xage < 20.000]
  [Xage < 18.000]
   [Xage < 17.000]
    [Xage < 13.000]
     [0]
     [Xage < 14.000]
      [Xavg_glucose_level < 122.310]
       [0]
       [0]
      [Xage < 16.000]
       [0]
       [0]
    [Xavg_glucose_level < 87.520]
     [Xavg_glucose_level < 67.810]
      [0]
      [0]
     [0]
   [Xage < 19.000]
    [Xavg_glucose_level < 111.380]
     [0]
     [0]
    [Xavg_glucose_level < 84.310]
     [0]
     [Xavg_glucose_level < 106.700]
      [0]
      [0]
  [Xage < 25.000]
   [Xage < 23.000]
    [Xage < 21.000]
     [Xavg_glucose_level < 84.620]
      [Xavg_glucose_level < 73.000]
       [0]
       [0]
      [Xavg_glucose_level < 96.570]
       [0]
       [Xavg_glucose_level < 104.480]
        [0]
        [Xavg_glucose_level < 184.250]
         [0]
         [0]
     [Xage < 22.000]
      [0]
      [Xavg_glucose_level < 105.220]
       [0]
       [Xavg_glucose_level < 130.340]
        [0]
        [0]
    [Xage < 24.000]
     [Xhypertension < 1.000]
      

In [214]:
test.head(1)

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked


In [236]:
root.predict(test[1:1])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [240]:
test[0:1]

,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
